In [1]:
import os
# Find the latest version of spark 3.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
spark_version = 'spark-3.0.2'
#spark_version = 'spark-3.<enter version>'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:11 http://security.ubuntu.com/ubuntu bionic-security/main amd64 Packages [1,964 kB]
Get:12 http://security.ubuntu.com/ubuntu bionic-security/universe amd64 Packa

In [2]:
#connect to postgres
!wget https://jdbc.postgresql.org/download/postgresql-42.2.19.jar

--2021-03-09 00:11:48--  https://jdbc.postgresql.org/download/postgresql-42.2.19.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1005078 (982K) [application/java-archive]
Saving to: ‘postgresql-42.2.19.jar’

postgresql-42.2.19. 100%[===================>] 981.52K  --.-KB/s    in 0.1s    

2021-03-09 00:11:49 (6.56 MB/s) - ‘postgresql-42.2.19.jar’ saved [1005078/1005078]



In [4]:
#create Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.19.jar").getOrCreate()

In [5]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url="https://sarahsupercoolbucket.s3.amazonaws.com/combineddata.csv"
spark.sparkContext.addFile(url)
project_df = spark.read.csv(SparkFiles.get("combineddata.csv"), sep=",", header=True, inferSchema=True)

# Show DataFrame
project_df.show()

+---+--------+--------------+-----------------+-------------+-----------------+----------------------------+------------------+-----------------------+------------------+-----------+-------------+--------------+-------------+-------------+--------------+----------------------+-----------------+-------------+-----------------+----------------------------+------------------+-----------------------+------------------+-----------+-------------+--------------+-------------+-------------+--------------+----------------------+-----------------+-------------+-----------------+----------------------------+------------------+-----------------------+------------------+-----------+-------------+--------------+-------------+-------------+--------------+----------------------+-----------------+-------------+-----------------+----------------------------+------------------+-----------------------+------------------+-----------+-------------+--------------+-------------+-------------+--------------+--

In [6]:
#rename index column to county_id
project_df = project_df.withColumnRenamed('_c0', 'county_id')

project_df.show()

+---------+--------+--------------+-----------------+-------------+-----------------+----------------------------+------------------+-----------------------+------------------+-----------+-------------+--------------+-------------+-------------+--------------+----------------------+-----------------+-------------+-----------------+----------------------------+------------------+-----------------------+------------------+-----------+-------------+--------------+-------------+-------------+--------------+----------------------+-----------------+-------------+-----------------+----------------------------+------------------+-----------------------+------------------+-----------+-------------+--------------+-------------+-------------+--------------+----------------------+-----------------+-------------+-----------------+----------------------------+------------------+-----------------------+------------------+-----------+-------------+--------------+-------------+-------------+-----------

In [7]:
#create new dataframe with county and state

county_df = project_df.select(['county_id', 'us_state', 'county'])
county_df.show()

+---------+--------+--------------+
|county_id|us_state|        county|
+---------+--------+--------------+
|        0|      AL|       Baldwin|
|        1|      AL|          Clay|
|        2|      AL|       Colbert|
|        3|      AL|        DeKalb|
|        4|      AL|        Elmore|
|        5|      AL|        Etowah|
|        6|      AL|       Houston|
|        7|      AL|     Jefferson|
|        8|      AL|      Lawrence|
|        9|      AL|       Madison|
|       10|      AL|        Mobile|
|       11|      AL|    Montgomery|
|       12|      AL|        Morgan|
|       13|      AL|       Russell|
|       14|      AL|        Shelby|
|       15|      AL|        Sumter|
|       16|      AL|    Tuscaloosa|
|       17|      AK|Aleutians East|
|       18|      AK|     Anchorage|
|       19|      AK|        Denali|
+---------+--------+--------------+
only showing top 20 rows



In [9]:
#create new df for yearly poverty
poverty_df = project_df.select('county_id', 'poverty_percentage2019', 'poverty_percentage2018', 'poverty_percentage2017', 'poverty_percentage2016', 'average_poverty_percentage')

poverty_df.show()

+---------+----------------------+----------------------+----------------------+----------------------+--------------------------+
|county_id|poverty_percentage2019|poverty_percentage2018|poverty_percentage2017|poverty_percentage2016|average_poverty_percentage|
+---------+----------------------+----------------------+----------------------+----------------------+--------------------------+
|        0|                  10.1|                   9.8|                  10.1|                  11.7|                    10.425|
|        1|                  16.6|                  17.6|                  18.1|                  18.9|                      17.8|
|        2|                  14.6|                  13.8|                  15.4|                  16.7|                    15.125|
|        3|                  18.2|                  21.3|                  19.5|                  20.5|                    19.875|
|        4|                  11.4|                  11.2|                 12.01|   

In [10]:
#create new df for 2019 day breakdown
day_count2019_df = project_df.select('county_id', 'days_with_aqi2019', 'good_days2019', 'moderate_days2019', 'unhealthy_sensitive_days2019', 'unhealthy_days2019', 'very_unhealthy_days2019', 'hazardous_days2019')

day_count2019_df.show()

+---------+-----------------+-------------+-----------------+----------------------------+------------------+-----------------------+------------------+
|county_id|days_with_aqi2019|good_days2019|moderate_days2019|unhealthy_sensitive_days2019|unhealthy_days2019|very_unhealthy_days2019|hazardous_days2019|
+---------+-----------------+-------------+-----------------+----------------------------+------------------+-----------------------+------------------+
|        0|              271|          237|               34|                           0|                 0|                      0|                 0|
|        1|              107|           97|               10|                           0|                 0|                      0|                 0|
|        2|              263|          252|               11|                           0|                 0|                      0|                 0|
|        3|              361|          324|               37|                     

In [11]:
#create new df for 2018 day breakdown
day_count2018_df = project_df.select('county_id', 'days_with_aqi2018', 'good_days2018', 'moderate_days2018', 'unhealthy_sensitive_days2018', 'unhealthy_days2018', 'very_unhealthy_days2018', 'hazardous_days2018')

day_count2018_df.show()

+---------+-----------------+-------------+-----------------+----------------------------+------------------+-----------------------+------------------+
|county_id|days_with_aqi2018|good_days2018|moderate_days2018|unhealthy_sensitive_days2018|unhealthy_days2018|very_unhealthy_days2018|hazardous_days2018|
+---------+-----------------+-------------+-----------------+----------------------------+------------------+-----------------------+------------------+
|        0|            270.0|        245.0|             25.0|                         0.0|               0.0|                    0.0|               0.0|
|        1|            110.0|        103.0|              7.0|                         0.0|               0.0|                    0.0|               0.0|
|        2|            277.0|        251.0|             26.0|                         0.0|               0.0|                    0.0|               0.0|
|        3|            350.0|        316.0|             34.0|                     

In [12]:
#create new df for 2017 day breakdown
day_count2017_df = project_df.select('county_id', 'days_with_aqi2017', 'good_days2017', 'moderate_days2017', 'unhealthy_sensitive_days2017', 'unhealthy_days2017', 'very_unhealthy_days2017', 'hazardous_days2017')

day_count2017_df.show()

+---------+-----------------+-------------+-----------------+----------------------------+------------------+-----------------------+------------------+
|county_id|days_with_aqi2017|good_days2017|moderate_days2017|unhealthy_sensitive_days2017|unhealthy_days2017|very_unhealthy_days2017|hazardous_days2017|
+---------+-----------------+-------------+-----------------+----------------------------+------------------+-----------------------+------------------+
|        0|            270.0|        241.0|             28.0|                         1.0|               0.0|                    0.0|               0.0|
|        1|            118.0|        104.0|             14.0|                         0.0|               0.0|                    0.0|               0.0|
|        2|            283.0|        265.0|             18.0|                         0.0|               0.0|                    0.0|               0.0|
|        3|            359.0|        329.0|             30.0|                     

In [13]:
#create new df for 2016 day breakdown
day_count2016_df = project_df.select('county_id', 'days_with_aqi2016', 'good_days2016', 'moderate_days2016', 'unhealthy_sensitive_days2016', 'unhealthy_days2016', 'very_unhealthy_days2016', 'hazardous_days2016')

day_count2016_df.show()

+---------+-----------------+-------------+-----------------+----------------------------+------------------+-----------------------+------------------+
|county_id|days_with_aqi2016|good_days2016|moderate_days2016|unhealthy_sensitive_days2016|unhealthy_days2016|very_unhealthy_days2016|hazardous_days2016|
+---------+-----------------+-------------+-----------------+----------------------------+------------------+-----------------------+------------------+
|        0|            279.0|        247.0|             32.0|                         0.0|               0.0|                    0.0|               0.0|
|        1|            116.0|        109.0|              7.0|                         0.0|               0.0|                    0.0|               0.0|
|        2|            282.0|        258.0|             23.0|                         1.0|               0.0|                    0.0|               0.0|
|        3|            348.0|        304.0|             43.0|                     

In [14]:
#create new df with pollutant breakdown 2019
pollutant_breakdown2019_df = project_df.select('county_id', 'days_co2019', 'days_no2_2019', 'days_ozone2019', 'days_so2_2019', 'days_pm2_2019', 'days_pm10_2019')

pollutant_breakdown2019_df.show()

+---------+-----------+-------------+--------------+-------------+-------------+--------------+
|county_id|days_co2019|days_no2_2019|days_ozone2019|days_so2_2019|days_pm2_2019|days_pm10_2019|
+---------+-----------+-------------+--------------+-------------+-------------+--------------+
|        0|          0|            0|           220|            0|           51|             0|
|        1|          0|            0|             0|            0|          107|             0|
|        2|          0|            0|           228|            0|           35|             0|
|        3|          0|            0|           331|            0|           30|             0|
|        4|          0|            0|           228|            0|            0|             0|
|        5|          0|            0|           218|            0|           61|             0|
|        6|          0|            0|           208|            0|           72|             0|
|        7|          0|            2|   

In [15]:
#create new df with pollutant breakdown 2018
pollutant_breakdown2018_df = project_df.select('county_id', 'days_co2018', 'days_no2_2018', 'days_ozone2018', 'days_so2_2018', 'days_pm2_2018', 'days_pm10_2018')

pollutant_breakdown2018_df.show()

+---------+-----------+-------------+--------------+-------------+-------------+--------------+
|county_id|days_co2018|days_no2_2018|days_ozone2018|days_so2_2018|days_pm2_2018|days_pm10_2018|
+---------+-----------+-------------+--------------+-------------+-------------+--------------+
|        0|        0.0|          0.0|         214.0|          0.0|         56.0|           0.0|
|        1|        0.0|          0.0|           0.0|          0.0|        110.0|           0.0|
|        2|        0.0|          0.0|         209.0|          0.0|         68.0|           0.0|
|        3|        0.0|          0.0|         317.0|          0.0|         33.0|           0.0|
|        4|        0.0|          0.0|         222.0|          0.0|          0.0|           0.0|
|        5|        0.0|          0.0|         179.0|          0.0|        145.0|           0.0|
|        6|        0.0|          0.0|         211.0|          0.0|         66.0|           0.0|
|        7|        0.0|          1.0|   

In [16]:
#create new df with pollutant breakdown 2017
pollutant_breakdown2017_df = project_df.select('county_id', 'days_co2017', 'days_no2_2017', 'days_ozone2017', 'days_so2_2017', 'days_pm2_2017', 'days_pm10_2017')

pollutant_breakdown2017_df.show()

+---------+-----------+-------------+--------------+-------------+-------------+--------------+
|county_id|days_co2017|days_no2_2017|days_ozone2017|days_so2_2017|days_pm2_2017|days_pm10_2017|
+---------+-----------+-------------+--------------+-------------+-------------+--------------+
|        0|        0.0|          0.0|         206.0|          0.0|         64.0|           0.0|
|        1|        0.0|          0.0|           0.0|          0.0|        118.0|           0.0|
|        2|        0.0|          0.0|         218.0|          0.0|         65.0|           0.0|
|        3|        0.0|          0.0|         315.0|          0.0|         44.0|           0.0|
|        4|        0.0|          0.0|         226.0|          0.0|          0.0|           0.0|
|        5|        0.0|          0.0|         133.0|          0.0|        227.0|           0.0|
|        6|        0.0|          0.0|         203.0|          0.0|         78.0|           0.0|
|        7|        0.0|          2.0|   

In [17]:
#create new df with pollutant breakdown 2016
pollutant_breakdown2016_df = project_df.select('county_id', 'days_co2016', 'days_no2_2016', 'days_ozone2016', 'days_so2_2016', 'days_pm2_2016', 'days_pm10_2016')

pollutant_breakdown2016_df.show()

+---------+-----------+-------------+--------------+-------------+-------------+--------------+
|county_id|days_co2016|days_no2_2016|days_ozone2016|days_so2_2016|days_pm2_2016|days_pm10_2016|
+---------+-----------+-------------+--------------+-------------+-------------+--------------+
|        0|        0.0|          0.0|         221.0|          0.0|         58.0|           0.0|
|        1|        0.0|          0.0|           0.0|          0.0|        116.0|           0.0|
|        2|        0.0|          0.0|         219.0|          0.0|         63.0|           0.0|
|        3|        0.0|          0.0|         321.0|          0.0|         27.0|           0.0|
|        4|        0.0|          0.0|         117.0|          0.0|          0.0|           0.0|
|        5|        0.0|          0.0|         104.0|          0.0|        248.0|           0.0|
|        6|        0.0|          0.0|         221.0|          0.0|         59.0|           0.0|
|        7|        0.0|         16.0|   

In [18]:
# create df with cancer data
cancer_df = project_df.select('county_id', 'percent_pop_per_100k', 'cancer_classification')
cancer_df.show()

+---------+--------------------+---------------------+
|county_id|percent_pop_per_100k|cancer_classification|
+---------+--------------------+---------------------+
|        0|         20.28698615|               medium|
|        1|         25.96356148|               medium|
|        2|         19.76146661|               medium|
|        3|          15.3574834|                  low|
|        4|         16.15852152|                  low|
|        5|         19.72566216|               medium|
|        6|         16.09452364|                  low|
|        7|         14.15427738|                  low|
|        8|          21.5049888|               medium|
|        9|         13.46218568|                  low|
|       10|         16.02894082|                  low|
|       11|         11.96010251|                  low|
|       12|         18.65477592|               medium|
|       13|         15.93614598|                  low|
|       14|         10.18467379|                  low|
|       15

In [19]:
#create df with average day count
avg_day_df = project_df.select('county_id', 'average_aqi_days', 'average_good_days', 'average_moderate_days', 'average_unhealthy_sensitive_days', 'average_unhealthy_days', 'average_very_unhealthy_days', 'average_hazardous_days')

avg_day_df.show()

+---------+----------------+-----------------+---------------------+--------------------------------+----------------------+---------------------------+----------------------+
|county_id|average_aqi_days|average_good_days|average_moderate_days|average_unhealthy_sensitive_days|average_unhealthy_days|average_very_unhealthy_days|average_hazardous_days|
+---------+----------------+-----------------+---------------------+--------------------------------+----------------------+---------------------------+----------------------+
|        0|           272.5|            242.5|                29.75|                            0.25|                   0.0|                        0.0|                   0.0|
|        1|          112.75|           103.25|                  9.5|                             0.0|                   0.0|                        0.0|                   0.0|
|        2|          276.25|            256.5|                 19.5|                            0.25|                   

In [20]:
#create table with average pollution breakdown
avg_pollutant_df = project_df.select('county_id', 'average_co_days', 'average_no2_days', 'average_ozone_days', 'average_so2_days', 'average_pm2_days', 'average_pm10_days')

avg_pollutant_df.show()

+---------+---------------+----------------+------------------+----------------+----------------+-----------------+
|county_id|average_co_days|average_no2_days|average_ozone_days|average_so2_days|average_pm2_days|average_pm10_days|
+---------+---------------+----------------+------------------+----------------+----------------+-----------------+
|        0|            0.0|             0.0|            215.25|             0.0|           57.25|              0.0|
|        1|            0.0|             0.0|               0.0|             0.0|          112.75|              0.0|
|        2|            0.0|             0.0|             218.5|             0.0|           57.75|              0.0|
|        3|            0.0|             0.0|             321.0|             0.0|            33.5|              0.0|
|        4|            0.0|             0.0|            198.25|             0.0|             0.0|              0.0|
|        5|            0.0|             0.0|             158.5|         

In [21]:
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://project.ci6yjllgzgzy.us-east-1.rds.amazonaws.com:5432/postgres"
config = {"user":"root", 
          "password": "BigFred3", 
          "driver":"org.postgresql.Driver"}

In [22]:
# Write DataFrames to tables in RDS
county_df.write.jdbc(url=jdbc_url, table='county', mode=mode, properties=config)

In [23]:
poverty_df.write.jdbc(url=jdbc_url, table='poverty', mode=mode, properties=config)

In [25]:
day_count2019_df.write.jdbc(url=jdbc_url, table='day_count2019', mode=mode, properties=config)

In [26]:
day_count2018_df.write.jdbc(url=jdbc_url, table='day_count2018', mode=mode, properties=config)

In [27]:
day_count2017_df.write.jdbc(url=jdbc_url, table='day_count2017', mode=mode, properties=config)

In [28]:
day_count2016_df.write.jdbc(url=jdbc_url, table='day_count2016', mode=mode, properties=config)

In [29]:
pollutant_breakdown2019_df.write.jdbc(url=jdbc_url, table='pollutant_breakdown2019', mode=mode, properties=config)

In [30]:
pollutant_breakdown2018_df.write.jdbc(url=jdbc_url, table='pollutant_breakdown2018', mode=mode, properties=config)

In [31]:
pollutant_breakdown2017_df.write.jdbc(url=jdbc_url, table='pollutant_breakdown2017', mode=mode, properties=config)

In [32]:
pollutant_breakdown2016_df.write.jdbc(url=jdbc_url, table='pollutant_breakdown2016', mode=mode, properties=config)

In [33]:
cancer_df.write.jdbc(url=jdbc_url, table='cancer_data', mode=mode, properties=config)

In [37]:
avg_day_df.write.jdbc(url=jdbc_url, table='avg_day', mode=mode, properties=config)

In [39]:
avg_pollutant_df.write.jdbc(url=jdbc_url, table='avg_pollutant', mode=mode, properties=config)